<a href="https://colab.research.google.com/github/gabrielaaguiv5/ProyectoFinal2/blob/main/notebooks/01_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================
#  SmartChat Insight - 01_cleaning.ipynb
#  Limpieza y estructuración de archivos .txt exportados de WhatsApp
# =========================================================

# 1️⃣ Importar librerías necesarias
import re
import pandas as pd
from pathlib import Path

# 2️⃣ Definir ruta de entrada y salida
raw_path = Path("data/raw_chats/")
clean_path = Path("data/cleaned/")
clean_path.mkdir(parents=True, exist_ok=True)

# 3️⃣ Función para leer y limpiar un archivo de chat
def parse_whatsapp_chat(file_path):
    """
    Convierte un archivo .txt exportado de WhatsApp en un DataFrame estructurado.
    Columnas: date, time, user, message
    """
    pattern = re.compile(r"^(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2}\s?[ap]\.?\s?m\.?) - (.*?): (.*)")
    data = []

    with open(file_path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            match = pattern.match(line)
            if match:
                date, time, user, message = match.groups()
                data.append([date, time, user.strip(), message.strip()])

    df = pd.DataFrame(data, columns=["date", "time", "user", "message"])
    return df

# 4️⃣ Procesar todos los archivos .txt en la carpeta data/raw_chats/
all_chats = []
for file in raw_path.glob("*.txt"):
    print(f"Procesando {file.name} ...")
    df_chat = parse_whatsapp_chat(file)
    df_chat["file_name"] = file.name
    all_chats.append(df_chat)

# 5️⃣ Unir todos los chats en un solo DataFrame
df_all = pd.concat(all_chats, ignore_index=True)

# 6️⃣ Eliminar mensajes del sistema (como “Los mensajes y las llamadas están cifrados...”)
df_all = df_all[~df_all["message"].str.contains("cifrados de extremo a extremo", na=False)]

# 7️⃣ Normalizar nombres y fechas
df_all["date"] = pd.to_datetime(df_all["date"], dayfirst=True, errors="coerce")
df_all["time"] = df_all["time"].str.replace(" ", " ")  # limpiar espacios Unicode

# 8️⃣ Guardar resultado limpio
output_file = clean_path / "whatsapp_clean.csv"
df_all.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ Archivo limpio guardado en: {output_file}")
print(f"Total de mensajes procesados: {len(df_all)}")

# 9️⃣ Vista previa
df_all.head(10)


ValueError: No objects to concatenate